In [2]:
# load results
import numpy as np

alexnet_predictions = np.load("alexnet_predictions.npz")["arr_0"]
alexnet_predictions

array([[3.94779863e-03, 2.13177186e-02, 2.82587498e-01, ...,
        2.13121027e-02, 5.10923739e-04, 7.79809547e-04],
       [3.17397341e-02, 2.27939729e-02, 5.96201345e-02, ...,
        2.25043856e-02, 8.88946839e-03, 9.63680912e-03],
       [1.03488244e-01, 1.84415221e-01, 7.55974829e-01, ...,
        6.58041053e-03, 3.66960512e-03, 1.04460865e-02],
       ...,
       [2.55904216e-02, 2.07660664e-02, 1.19247288e-01, ...,
        9.75004956e-03, 8.14527366e-03, 3.96740716e-03],
       [1.05909910e-02, 5.22454968e-03, 8.89304131e-02, ...,
        1.62097216e-02, 8.68206099e-03, 3.23032681e-03],
       [5.39686829e-02, 1.37956887e-02, 5.94881102e-02, ...,
        1.52153283e-01, 3.16583016e-03, 3.53303924e-03]], dtype=float32)

In [3]:
import pandas as pd


from dataset import PosterDataset

df = pd.read_csv(PosterDataset.csv_file)
true_values = df.iloc[:, -20:-1].values
genres = df.columns[-20:-1]
genres = list(map(lambda x: x[1:], genres))
single_genres = df.genre_count == 1

In [4]:
from sklearn.metrics import accuracy_score, f1_score

# calculate accuracy
single_prediction = (
    alexnet_predictions[single_genres]
    == alexnet_predictions.max(axis=1)[single_genres][:, None]
)
accuracy_score(single_prediction, true_values[single_genres]), f1_score(
    single_prediction, true_values[single_genres], average="micro"
)

(0.43490739854886845, 0.4349073985488685)

### 3 Single Label


In [5]:
three_columns = ["Action", "Romance", "Thriller"]
three_genres = np.isin(genres, three_columns)
three_genres_index_mask = (
    true_values[:, three_genres].sum(axis=1) >= 1
) & single_genres
three_genres_index_mask.sum()

15976

In [6]:
from sklearn.metrics import accuracy_score, f1_score

# calculate accuracy
three_single_prediction = alexnet_predictions[three_genres_index_mask,][:, three_genres]
three_single_prediction_classes = (
    three_single_prediction == three_single_prediction.max(axis=1)[:, None]
)

accuracy_score(
    three_single_prediction_classes,
    true_values[three_genres_index_mask,][:, three_genres],
), f1_score(
    three_single_prediction_classes,
    true_values[three_genres_index_mask,][:, three_genres],
    average="samples",
)

(0.6229969954932398, 0.6229969954932398)

In [7]:
three_single_prediction_classes.sum(axis=0) / three_single_prediction.shape[0]

array([0.25776164, 0.41612419, 0.32611417])